# Import lib's

In [7]:
import pandas as pd
import os as os
import subprocess
from IPython.display import display, Markdown

# Location's

Below are the file path definitions required for the Script. These variables tell the script where to find the software, the 3D environment, the logic, and the settings.


`BLENDER_EXE`: The absolute file path to the Blender application executable (.exe) on your computer.

`BLEND_FILE`: The path to the .blend project file that acts as the base template for the simulation.

`EXCEL_FILE`: The path to the Excel workbook

`SENSOR_PROGRAM_FILE`: The path to the external Python script containing the Blender-specific logic

`DATA_SAVE_LOCATION`: The location where the generated data is saved

In [8]:
# User specific
BLENDER_EXE = r"C:\Program Files\Blender Foundation\Blender 5.0\blender.exe"

PROJECT_DIR             = os.getcwd()

# Project specific
BLEND_FILE              = os.path.join(PROJECT_DIR, r"..\Blender_Generated_Data\DataGenerator.blend")
EXCEL_FILE              = os.path.join(PROJECT_DIR, r"Data_generation_settings.xlsx")
SENSOR_PROGRAM_FILE     = os.path.join(PROJECT_DIR, r"BlenderSensorProgram.py")

OUTPUT_ROOT             = os.path.join(PROJECT_DIR, r"..\Blender_Generated_Data")

print(f"Project dir: {PROJECT_DIR}")
print(f"Blender file location: {BLEND_FILE}")

Project dir: c:\Users\RobinSchool\Stichting Hogeschool Utrecht\MNLE Imagine Project - Documents\DataProgram\Program's
Blender file location: c:\Users\RobinSchool\Stichting Hogeschool Utrecht\MNLE Imagine Project - Documents\DataProgram\Program's\..\Blender_Generated_Data\DataGenerator.blend


# Vallidating the paths

In [9]:
critical_files = {
    "Blender Application": BLENDER_EXE,
    "Excel Configuration": EXCEL_FILE,
    "Blender Scene File": BLEND_FILE,
    "Blender Python Script": SENSOR_PROGRAM_FILE
}

errors =0

    
# 1. Check Critical Files
for name, path in critical_files.items():
    if not os.path.exists(path):
        # Only print if NOT found
        print(f"❌ MISSING FILE: {name}")
        print(f"   -> Searched at: {path}")
        errors+=1

if errors:
    raise Exception(f"{errors} wrong file paths")



try:
    if not os.path.exists(OUTPUT_ROOT):
        os.makedirs(OUTPUT_ROOT)
except Exception as e:
    print(f"❌ ERROR: Could not create Output directory: {OUTPUT_ROOT}")
    print(f"   -> Details: {e}")
    all_good = False


if errors:
    raise Exception("Output couln't be created")



# Exel 

In [10]:
def clean_cols(df):
    df.columns = df.columns.str.replace(r'\s*\(.*\)', '', regex=True)
    df.columns = df.columns.str.strip()
    return df

def load_excel_data():
    if not os.path.exists(EXCEL_FILE):
        print(f"File not found: {EXCEL_FILE}")
        return
    
    print(f"--- Loading {EXCEL_FILE} ---")

    # ==========================================
    # STEP 1: Load Definitions (Tables)
    # ==========================================
    try:
        # Load Sensors (Sheet 'Sensors', index 'sensor_id')
        df_sensors = pd.read_excel(EXCEL_FILE, sheet_name='Sensors')
        df_sensors = clean_cols(df_sensors)
        df_sensors.set_index('sensor_id', inplace=True)
        
        # Load Positions (Sheet 'Pos', index 'config_id')
        df_positions = pd.read_excel(EXCEL_FILE, sheet_name='Pos')
        df_positions = clean_cols(df_positions)
        df_positions.set_index('config_id', inplace=True)
        
        # Load Datasets (Sheet 'Data', index 'dataset_name')
        df_datasets = pd.read_excel(EXCEL_FILE, sheet_name='Data')
        df_datasets.set_index('dataset_name', inplace=True)
        
        print("Definitions loaded:")
        print(f"   - {len(df_sensors)} Sensors")
        print(f"   - {len(df_positions)} Positions")
        print(f"   - {len(df_datasets)} Datasets")

    except Exception as e:
        print(f"ERROR loading definitions: {e}")
        return

    # ==========================================
    # STEP 2: Load the Matrix
    # ==========================================
    try:
        df_matrix = pd.read_excel(EXCEL_FILE, sheet_name='Run_Matrix')
        
        # Cleanup: Drop rows where 'Batch_ID' is empty
        df_matrix = df_matrix.dropna(subset=['Batch_ID'])
        
        # 1. FIX: Filter on the 'Generate' column, not Batch_ID
        # We assume the column is named 'Generate' as shown in the image
        # active_batches = df_matrix[df_matrix['Generate'].astype(str).str.upper().isin(['JA', 'YES', 'TRUE'])]
        
        # print(f"Matrix loaded: {len(active_batches)} active batches found.")

    except Exception as e:
        print(f"ERROR loading Matrix: {e}")
        return None ,None ,None , None
  
    return df_sensors, df_datasets, df_positions,df_matrix
        



# launc blender

In [11]:
def run_blender(sensor_spec,pos_spec,data_spec,output_dir):
    cmd = [
        BLENDER_EXE,
        "-b", BLEND_FILE,            # Run in background (no UI)
        "-P", SENSOR_PROGRAM_FILE,   # Run the python script
        "--",                        # Separator (args after this are for our script)
        
        # Pass arguments (convert numbers to strings!)
        "--sensor_res", f"{int(sensor_spec['resolution_width'])}x{int(sensor_spec['resolution_height'])}",
        "--sensor_fov", f"{sensor_spec['fov_horizontal']}x{sensor_spec['fov_vertical']}",
        "--position", f"{pos_spec['pos_x']},{pos_spec['pos_y']},{pos_spec['pos_z']}",
        "--samples", str(int(data_spec['num_samples'])),
        "--output", output_dir,
        
        # Optional args (using .get() in case they are missing in Excel)
        "--noise", str(sensor_spec.get('noise_level', 0.0)),
        "--target_name", "Cube"
    ]

    # Add ranges if they exist in the dataset spec
    if 'rot_range' in data_spec:
        cmd.extend(["--rot_range", str(data_spec['rot_range'])])
    if 'trans_range' in data_spec:
        cmd.extend(["--trans_range", str(data_spec['trans_range'])])

    # 4. Execute
    # print(f"      >>> Launching Blender for batch: ...")
    try:
        # check=True will raise an error if Blender crashes
        subprocess.run(cmd, check=True) 
        # print("      >>> Done.")
    except subprocess.CalledProcessError as e:
        print(f"      !!! Blender Error: {e}")

In [12]:
ERROR_REPORT_MODE = 'SUMMARY' 

# 1. Load Data
df_sensors, df_datasets, df_positions, df_matrix = load_excel_data()

# 2. Filter active batches
active_batches = df_matrix[df_matrix['Generate'].astype(str).str.upper().isin(['JA', 'YES', 'TRUE'])]
print(f"Matrix loaded: {len(active_batches)} active batches found.")

# Get column names
matrix_cols = df_matrix.columns.tolist()

# --- STATUS TRACKING ---
batch_statuses = {}  # Display text
batch_errors = {}    # Store exceptions: { 'Batch_ID': [list of errors] }

# Pre-fill dictionary
for idx, row in active_batches.iterrows():
    b_id = row['Batch_ID']
    batch_statuses[b_id] = "Waiting..."
    batch_errors[b_id] = []

# Function to refresh the screen
def update_status_display(handle, statuses):
    lines = ["### Batch Status Overview"]
    for b_id, status in statuses.items():
        # Bold text if running or if finished with errors
        if "Running" in status:
            lines.append(f"- **{b_id}**: {status}")
        elif "errors" in status.lower():
            lines.append(f"- **{b_id}** (ATTENTION): {status}")
        else:
            lines.append(f"- {b_id}: {status}")
    
    handle.update(Markdown("\n".join(lines)))

# Create display
status_handle = display(Markdown("Initializing..."), display_id=True)
update_status_display(status_handle, batch_statuses)

# --- START PROCESSING ---

for idx, row in active_batches.iterrows():
    batch_id = row['Batch_ID']
    dataset_id = row['Data generation settings']

    # Initialize counters for this batch
    success_count = 0
    fail_count = 0
    
    # Validation
    if dataset_id not in df_datasets.index:
        batch_statuses[batch_id] = f"ERROR: Dataset '{dataset_id}' not found!"
        update_status_display(status_handle, batch_statuses)
        continue

    # Select Sensors & Positions
    selected_sensors = []
    selected_positions = []

    for col_name in matrix_cols:
        if col_name in ['Batch_ID', 'Generate', 'Data generation settings']:
            continue
        
        cell_value = row[col_name]
        if pd.notna(cell_value) and str(cell_value).strip() != "":
            if col_name in df_sensors.index:
                selected_sensors.append(col_name)
            elif col_name in df_positions.index:
                selected_positions.append(col_name)

    if not selected_sensors or not selected_positions:
        batch_statuses[batch_id] = "Skipped (No sensors/positions selected)"
        update_status_display(status_handle, batch_statuses)
        continue

    # --- START LOOPS ---
    total_steps = len(selected_sensors) * len(selected_positions)
    current_step = 0
    
    for sens_id in selected_sensors:
        sensor_spec = df_sensors.loc[sens_id]
        
        for pos_id in selected_positions:
            pos_spec = df_positions.loc[pos_id]
            data_spec = df_datasets.loc[dataset_id]
            
            current_step += 1
            
            # Construct Status Message
            # Shows: Running [1/10] | Success: 0 | Failed: 0
            status_msg = (f"Running [{current_step}/{total_steps}] | "
                          f"Success: {success_count} | Failed: {fail_count} | "
                          f"Current: {sens_id} @ {pos_id}")
            
            batch_statuses[batch_id] = status_msg
            update_status_display(status_handle, batch_statuses)

            dir_name = os.path.join(OUTPUT_ROOT, batch_id, sens_id, pos_id)
            
            # === BLENDER CALL ===
            try:
                run_blender(sensor_spec, pos_spec, data_spec, dir_name)
                success_count += 1
            except Exception as e:
                fail_count += 1
                error_msg = f"{sens_id}@{pos_id}: {str(e)}"
                batch_errors[batch_id].append(error_msg)

    # --- FINALIZE BATCH STATUS ---
    if fail_count > 0:
        batch_statuses[batch_id] = f"Finished with {fail_count} errors (Success: {success_count})"
    else:
        batch_statuses[batch_id] = f"Completed successfully ({success_count} runs)"
    
    update_status_display(status_handle, batch_statuses)

# --- FINAL ERROR REPORT ---
display(Markdown("--- **All Batches Finished** ---"))

total_errors = sum(len(errs) for errs in batch_errors.values())

if total_errors > 0:
    print(f"\n=== ERROR REPORT (Mode: {ERROR_REPORT_MODE}) ===")
    
    for b_id, errors in batch_errors.items():
        if not errors:
            continue
            
        print(f"\nBatch: {b_id} ({len(errors)} failures)")
        
        if ERROR_REPORT_MODE == 'SUMMARY':
            # Only print unique errors to avoid spamming the same message 100 times
            # We filter duplicates by converting the list to a set
            unique_errors = list(set(errors))
            for i, err in enumerate(unique_errors, 1):
                print(f"  {i}. {err}")
            if len(errors) > len(unique_errors):
                print(f"  ... (and {len(errors) - len(unique_errors)} duplicates)")
                
        else:
            # Print ALL errors
            for i, err in enumerate(errors, 1):
                print(f"  {i}. {err}")
else:
    print("\nNo errors occurred during generation.")

--- Loading c:\Users\RobinSchool\Stichting Hogeschool Utrecht\MNLE Imagine Project - Documents\DataProgram\Program's\Data_generation_settings.xlsx ---
Definitions loaded:
   - 5 Sensors
   - 3 Positions
   - 4 Datasets
Matrix loaded: 2 active batches found.


### Batch Status Overview
- Test_1: Completed successfully (1 runs)
- Test_2: Completed successfully (1 runs)

--- **All Batches Finished** ---


No errors occurred during generation.
